# NLP Lab 7: Dependency Parsing

**Mục tiêu:** Thực hành phân tích cú pháp phụ thuộc (Dependency Parsing) với spaCy.

**Nội dung:**
1. Phần 1: Giới thiệu và Cài đặt
2. Phần 2: Phân tích câu và Trực quan hóa
3. Phần 3: Truy cập các thành phần trong cây phụ thuộc
4. Phần 4: Duyệt cây phụ thuộc để trích xuất thông tin
5. Phần 5: Bài tập tự luyện

In [1]:
import spacy
from spacy import displacy

# Tải mô hình tiếng Anh
nlp = spacy.load("en_core_web_md")

## Phần 1: Giới thiệu

**Phân tích cú pháp phụ thuộc** là kỹ thuật hiểu cấu trúc ngữ pháp của câu dưới dạng các mối quan hệ:
- **Head (điều khiển)**: Từ chính trong quan hệ
- **Dependent (phụ thuộc)**: Từ bổ nghĩa cho head

Ví dụ: "The quick fox" → "fox" là head, "The" và "quick" là dependents.

## Phần 2: Phân tích câu và Trực quan hóa

In [2]:
# Câu ví dụ
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

print(f"Câu: {text}")
print(f"Số token: {len(doc)}")

Câu: The quick brown fox jumps over the lazy dog.
Số token: 10


In [3]:
# Trực quan hóa cây phụ thuộc (trong Jupyter)
displacy.render(doc, style="dep", jupyter=True, options={"compact": True})

In [4]:
# Phân tích cây phụ thuộc
print("Phân tích cây phụ thuộc:")
print(f"{'TEXT':<10} | {'DEP':<10} | {'HEAD':<10} | {'POS':<8}")
print("-" * 45)
for token in doc:
    print(f"{token.text:<10} | {token.dep_:<10} | {token.head.text:<10} | {token.pos_:<8}")

Phân tích cây phụ thuộc:
TEXT       | DEP        | HEAD       | POS     
---------------------------------------------
The        | det        | fox        | DET     
quick      | amod       | fox        | ADJ     
brown      | amod       | fox        | ADJ     
fox        | nsubj      | jumps      | NOUN    
jumps      | ROOT       | jumps      | VERB    
over       | prep       | jumps      | ADP     
the        | det        | dog        | DET     
lazy       | amod       | dog        | ADJ     
dog        | pobj       | over       | NOUN    
.          | punct      | jumps      | PUNCT   


### Câu hỏi phân tích:
- **ROOT của câu**: "jumps" (động từ chính)
- **Dependents của "jumps"**: fox (nsubj), over (prep)
- **"fox" là head của**: The (det), quick (amod), brown (amod)

## Phần 3: Truy cập các thành phần trong cây phụ thuộc

In [5]:
# Câu phức tạp hơn
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)

print(f"{'TEXT':<12} | {'DEP':<10} | {'HEAD TEXT':<12} | {'HEAD POS':<8} | {'CHILDREN'}")
print("-" * 70)
for token in doc:
    children = [child.text for child in token.children]
    print(f"{token.text:<12} | {token.dep_:<10} | {token.head.text:<12} | {token.head.pos_:<8} | {children}")

TEXT         | DEP        | HEAD TEXT    | HEAD POS | CHILDREN
----------------------------------------------------------------------
Apple        | nsubj      | looking      | VERB     | []
is           | aux        | looking      | VERB     | []
looking      | ROOT       | looking      | VERB     | ['Apple', 'is', 'at']
at           | prep       | looking      | VERB     | ['buying']
buying       | pcomp      | at           | ADP      | ['startup']
U.K.         | compound   | startup      | NOUN     | []
startup      | dobj       | buying       | VERB     | ['U.K.', 'for']
for          | prep       | startup      | NOUN     | ['billion']
$            | quantmod   | billion      | NUM      | []
1            | compound   | billion      | NUM      | []
billion      | pobj       | for          | ADP      | ['$', '1']


### Giải thích các thuộc tính:
- `token.text`: Văn bản của token
- `token.dep_`: Nhãn quan hệ phụ thuộc với head
- `token.head`: Token head (điều khiển)
- `token.children`: Iterator các token con (dependent)

## Phần 4: Duyệt cây phụ thuộc để trích xuất thông tin

### 4.1. Tìm chủ ngữ và tân ngữ của động từ

In [6]:
text = "The cat chased the mouse and the dog watched them."
doc = nlp(text)

print("Tìm bộ ba (Chủ ngữ, Động từ, Tân ngữ):")
print("-" * 40)

for token in doc:
    if token.pos_ == "VERB":
        verb = token.text
        subject = ""
        obj = ""
        
        for child in token.children:
            if child.dep_ == "nsubj":
                subject = child.text
            if child.dep_ == "dobj":
                obj = child.text
        
        if subject and obj:
            print(f"Found Triplet: ({subject}, {verb}, {obj})")

Tìm bộ ba (Chủ ngữ, Động từ, Tân ngữ):
----------------------------------------
Found Triplet: (cat, chased, mouse)
Found Triplet: (dog, watched, them)


### 4.2. Tìm các tính từ bổ nghĩa cho danh từ

In [7]:
text = "The big, fluffy white cat is sleeping on the warm mat."
doc = nlp(text)

print("Tìm tính từ bổ nghĩa cho danh từ:")
print("-" * 50)

for token in doc:
    if token.pos_ == "NOUN":
        adjectives = []
        for child in token.children:
            if child.dep_ == "amod":
                adjectives.append(child.text)
        
        if adjectives:
            print(f"Danh từ '{token.text}' được bổ nghĩa bởi: {adjectives}")

Tìm tính từ bổ nghĩa cho danh từ:
--------------------------------------------------
Danh từ 'cat' được bổ nghĩa bởi: ['big', 'fluffy', 'white']
Danh từ 'mat' được bổ nghĩa bởi: ['warm']


## Phần 5: Bài tập tự luyện

### Bài 1: Tìm động từ chính của câu

In [8]:
def find_main_verb(doc):
    """Tìm động từ chính (ROOT) của câu."""
    for token in doc:
        if token.dep_ == "ROOT":
            return token
    return None

# Test
test_sentences = [
    "The cat is sleeping on the mat.",
    "She quickly finished her homework.",
    "The students are studying for the exam."
]

print("Bài 1: Tìm động từ chính")
print("-" * 50)
for sent in test_sentences:
    doc = nlp(sent)
    main_verb = find_main_verb(doc)
    print(f"Câu: '{sent}'")
    print(f"  → Động từ chính: '{main_verb.text}' (POS: {main_verb.pos_})\n")

Bài 1: Tìm động từ chính
--------------------------------------------------
Câu: 'The cat is sleeping on the mat.'
  → Động từ chính: 'sleeping' (POS: VERB)

Câu: 'She quickly finished her homework.'
  → Động từ chính: 'finished' (POS: VERB)

Câu: 'The students are studying for the exam.'
  → Động từ chính: 'studying' (POS: VERB)



### Bài 2: Trích xuất các cụm danh từ (Noun Chunks)

In [9]:
def extract_noun_chunks(doc):
    """Trích xuất cụm danh từ từ câu."""
    chunks = []
    for token in doc:
        if token.pos_ == "NOUN":
            # Thu thập các từ bổ nghĩa (det, amod, compound)
            chunk_tokens = []
            for child in token.children:
                if child.dep_ in ["det", "amod", "compound"]:
                    chunk_tokens.append(child)
            chunk_tokens.append(token)
            # Sắp xếp theo vị trí trong câu
            chunk_tokens.sort(key=lambda x: x.i)
            chunk_text = " ".join([t.text for t in chunk_tokens])
            chunks.append(chunk_text)
    return chunks

# Test
text = "The big brown dog chased the small white cat."
doc = nlp(text)

print("Bài 2: Trích xuất cụm danh từ")
print("-" * 50)
print(f"Câu: '{text}'")
print(f"\nCách 1 - Tự implement:")
for chunk in extract_noun_chunks(doc):
    print(f"  → {chunk}")

print(f"\nCách 2 - Dùng spaCy noun_chunks:")
for chunk in doc.noun_chunks:
    print(f"  → {chunk.text}")

Bài 2: Trích xuất cụm danh từ
--------------------------------------------------
Câu: 'The big brown dog chased the small white cat.'

Cách 1 - Tự implement:
  → The big brown dog
  → the small white cat

Cách 2 - Dùng spaCy noun_chunks:
  → The big brown dog
  → the small white cat


### Bài 3: Tìm đường đi ngắn nhất đến ROOT

In [10]:
def get_path_to_root(token):
    """Tìm đường đi từ token đến ROOT."""
    path = [token]
    while token.dep_ != "ROOT":
        token = token.head
        path.append(token)
    return path

# Test
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

print("Bài 3: Đường đi đến ROOT")
print("-" * 50)
print(f"Câu: '{text}'\n")

for token in doc:
    if token.text in ["quick", "dog", "lazy"]:
        path = get_path_to_root(token)
        path_str = " → ".join([f"{t.text}({t.dep_})" for t in path])
        print(f"Từ '{token.text}' đến ROOT: {path_str}")

Bài 3: Đường đi đến ROOT
--------------------------------------------------
Câu: 'The quick brown fox jumps over the lazy dog.'

Từ 'quick' đến ROOT: quick(amod) → fox(nsubj) → jumps(ROOT)
Từ 'lazy' đến ROOT: lazy(amod) → dog(pobj) → over(prep) → jumps(ROOT)
Từ 'dog' đến ROOT: dog(pobj) → over(prep) → jumps(ROOT)
